<a href="https://colab.research.google.com/github/cksdlakstp12/izone_finder/blob/main/IZONE_FINDER_annotation_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# this file for yolo model train

In [ ]:
!pip install mtcnn

import numpy as np
import pickle
import zipfile
import time
import os

import cv2
import matplotlib.pyplot as plt
from PIL import Image

import mtcnn
import tensorflow as tf
from mtcnn.mtcnn import MTCNN

from google.colab import drive
drive.mount("/content/drive")

     |████████████████████████████████| 2.3 MB 21.1 MB/s 
Mounted at /content/drive


In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error : Creating directory. " + directory)

In [ ]:
FILTERING_THRES = 0.95

DRIVE_IMAGE_PATH = "/content/drive/MyDrive/IZONE_finder/stage_dataset/image"
DRIVE_ANNOTATION_PATH = "/content/drive/MyDrive/IZONE_finder/stage_dataset/annotation"

LOCAL_IMAGE_PATH = "/content/image"
LOCAL_ANNOTATION_PATH = "/content/annotation"

MODEL_NAME = "member_classification_model_B7"
MODEL_PATH = f"/content/drive/MyDrive/IZONE_finder/IZ_ONE_Finder/weights/{MODEL_NAME}.h5"

ZIPFILE_PATH = os.path.join(DRIVE_IMAGE_PATH, "stage_images.zip")

In [ ]:
createFolder(LOCAL_IMAGE_PATH)
createFolder(LOCAL_ANNOTATION_PATH)

stage_image = zipfile.ZipFile(ZIPFILE_PATH)
stage_image.extractall(LOCAL_IMAGE_PATH)
stage_image.close()

In [ ]:
classifier = tf.keras.models.load_model(MODEL_PATH)
face_detector = MTCNN()

In [ ]:
for image_name in os.listdir(LOCAL_IMAGE_PATH):
  image_path = os.path.join(LOCAL_IMAGE_PATH, image_name)
  image_np = cv2.imread(image_path)

  img_height, img_width = image_np.shape

  detected_faces = face_detector.detect_faces(image_np)

  result_list = list()
  
  if not detected_faces:
    !rm image_path
    continue

  for face_info in detected_faces:
    x1, y1, width, height = face_info["box"]
    x2, y2 = x1 + width, y1 + height

    face = img[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize((224, 224))
    face_array = np.asarray(image)

    member = classifier.predict([face_array])
    member_index = member.argmax()

    if member.max() > FILTERING_THRES:
      result = [member_index, x1, y1, x2, y2] # w, h norm?
      # result = [member_index, x1/img_width, y1/img_height, x2/img_width, y2/img_height]
      # result = [member_index, x1, y1, width, height]
      result_list.append(result)

  with open(f"{LOCAL_ANNOTATION_PATH}/{image_name.split('.')[0]}.pkl", "wb") as f:
    pickle.dump(np.array(result_list), f)

SyntaxError: ignored

In [ ]:
with zipfile.ZipFile(f'/content/stage_images.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
  image_name_list = os.listdir(LOCAL_IMAGE_PATH)
    
  for image_name in image_name_list:
    image_path = os.path.join(LOCAL_IMAGE_PATH, image_name)
    new_zip.write(image_path, arcname=image_name)

  train_split = len(image_name_list) * 0.75
  test_split = train_split + len(image_name_list) * 0.15

  # write train data
  with open("train.txt", "w") as f:
    for image_name in image_name_list[:train_split]:
      writing_image_path = os.path.join(LOCAL_IMAGE_PATH, image_name)
      f.write(writing_image_path)

  # write test data
  with open("test.txt", "w") as f:
    for image_name in image_name_list[train_split:test_split]:
      writing_image_path = os.path.join(LOCAL_IMAGE_PATH, image_name)
      f.write(writing_image_path)
    
  # write validation data
  with open("validation.txt", "w") as f:
    for image_name in image_name_list[test_split:]:
      writing_image_path = os.path.join(LOCAL_IMAGE_PATH, image_name)
      f.write(writing_image_path)

with zipfile.ZipFile(f'/content/stage_annotations.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
  annotation_name_list = os.listdir(LOCAL_ANNOTATION_PATH)
  for annotation_name in annotation_name_list:
    anno_path = os.path.join(LOCAL_ANNOTATION_PATH, annotation_name)
    new_zip.write(anno_path, arcname=annotation_name)

!cp /content/stage_images.zip {DRIVE_IMAGE_PATH}
!cp /content/stage_annotations.zip {DRIVE_ANNOTATION_PATH}

!cp /content/train.txt {DRIVE_TXT_PATH}
!cp /content/test.txt {DRIVE_TXT_PATH}
!cp /content/validation.txt {DRIVE_TXT_PATH}

print("All files successfully copied to drive")